## Parameters

In [16]:
batch_size = 3
seq_length = 4
rnn_size   = 10
vocab_size = 5
save_dir_GAN = 'models_GAN'
vocab_file = 'simple_vocab.pkl'
model = 'lstm'
num_layers = 2

## Generate Samples

In [7]:
def load_vocab(save_dir_GAN, vocab_file):
    '''Load vocabulary objects.
    
    Args:
        save_dir_GAN:  Directory containing vocab_files.
        vocab_file: Vocab file to use.
    '''
    with open(os.path.join(save_dir_GAN, vocab_file)) as f:
            chars, vocab = cPickle.load(f)
    return chars, vocab

# Return text from indices
def map_to_text(indices, chars):
    text = []
    for index in indices:
        text.append(chars[index])
    return text

In [20]:
import tensorflow as tf
import os
import cPickle
from tensorflow.contrib.distributions import Categorical
tf.reset_default_graph()

def generate():    
    # Initial indices
    batch_indices = tf.constant(3, shape=[batch_size])

    # RNN
    outputs, indices = [], []
    cell = tf.nn.rnn_cell.BasicRNNCell(rnn_size)
    state = cell.zero_state(batch_size, tf.float32)
        
    with tf.variable_scope('rnn'):
        # Embeddings and Logits
        embedding = tf.get_variable('embedding', [vocab_size, rnn_size])
        softmax   = tf.get_variable('softmax', [rnn_size, vocab_size])

        inp = tf.nn.embedding_lookup(embedding, batch_indices)
        for i in xrange(seq_length):
            indices.append(batch_indices)
            if i > 0:
                tf.get_variable_scope().reuse_variables()
            rnn_out, state = cell(inp, state)
            logits_out = tf.matmul(rnn_out, softmax)
            outputs.append(logits_out)
            batch_indices = tf.squeeze(Categorical(logits_out).sample(n=1))
            inp = tf.nn.embedding_lookup(embedding, batch_indices)
  
    return outputs, indices
            

with tf.Session() as sess: 
    outputs, indices = generate()
    init_op = tf.initialize_all_variables()
    sess.run(init_op)

    chars, vocab = load_vocab(save_dir_GAN, vocab_file)
    
    for i, output in enumerate(outputs):
        print 'Iteration %d'%i
        print sess.run(output),'\n'
    
    for i, batch_indices in enumerate(indices):
        print 'Iteration %d'%i
        indices_eval = sess.run(batch_indices)
        print map_to_text(indices_eval, chars),'\n'
    
#     for line in indices:
#         print ''.join(line)

Iteration 0
[[ 0.27288669 -0.28613415 -0.45236325 -0.58369422 -0.08270382]
 [ 0.27288669 -0.28613415 -0.45236325 -0.58369422 -0.08270382]
 [ 0.27288669 -0.28613415 -0.45236325 -0.58369422 -0.08270382]] 

Iteration 1
[[ 0.16775502 -0.38674468 -0.20835336 -0.65464985 -0.417173  ]
 [-0.02882031 -0.45022762  0.57098377 -0.37166816 -0.27681977]
 [-0.02638191 -0.16726628  0.33778578 -0.56056243 -0.24883367]] 

Iteration 2
[[ 0.27418253 -0.75881737 -0.13004506 -0.5971753  -1.05401576]
 [ 0.52052826 -0.57565653  0.14307176 -0.53279293 -0.22682998]
 [-0.5994361   0.29144675  0.5674336   0.37106675 -0.25375479]] 

Iteration 3
[[ 0.12238991 -0.25080177 -0.52532065 -0.65432006 -0.0159961 ]
 [-0.98164564  0.05685884  0.32164502  0.25036573 -0.96554631]
 [ 0.2813935  -0.71913826 -0.27651945 -0.59287626 -0.76545358]] 

Iteration 0
['b', 'b', 'b'] 

Iteration 1
['\n', 'r', '\n'] 

Iteration 2
['b', ' ', 'b'] 

Iteration 3
['r', 'e', 'b'] 



## Train Generator

In [30]:
from tensorflow.python.ops.nn import rnn_cell
from tensorflow.python.ops.nn import rnn
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops

tf.reset_default_graph()

def construct_gan():
    if model == 'rnn':
        cell_gen = rnn_cell.BasicRNNCell(rnn_size)
        cell_dis = rnn_cell.BasicRNNCell(rnn_size)
    elif model == 'gru':
        cell_gen = rnn_cell.GRUCell(rnn_size)
        cell_dis = rnn_cell.GRUCell(rnn_size)
    elif model == 'lstm':
        cell_gen = rnn_cell.BasicLSTMCell(rnn_size, state_is_tuple=False)
        cell_dis = rnn_cell.BasicLSTMCell(rnn_size, state_is_tuple=False)
    else:
        raise NotImplementedError('Model type not supported: {}'
                                  .format(model))

    # Initial indices.
    batch_indices = tf.constant(0, shape=[batch_size])
        
    # Generator Portion of GAN.
    with tf.variable_scope('generator'):
        outputs_gen, logit_sequence = [], []
        cell_gen = rnn_cell.MultiRNNCell([cell_gen] * num_layers)
        state_gen = cell_gen.zero_state(batch_size, tf.float32)

        with tf.variable_scope('rnn'):
            softmax_w = tf.get_variable('softmax_w', [rnn_size, vocab_size])
            softmax_b = tf.get_variable('softmax_b', [vocab_size])
            embedding = tf.get_variable('embedding', [vocab_size, rnn_size])
            inp = tf.nn.embedding_lookup(embedding, batch_indices)
    
            for i in xrange(seq_length):
                if i > 0:
                    tf.get_variable_scope().reuse_variables()
              
                # RNN.
                output_gen, state_gen = cell_gen(inp, state_gen)
                logits_gen = tf.nn.xw_plus_b(output_gen, softmax_w, 
                                             softmax_b)
                
                # Sampling.
                sample_op = tf.stop_gradient(Categorical(
                                            logits_gen).sample(n=1))
                batch_indices = tf.squeeze(sample_op)
                inp = tf.nn.embedding_lookup(embedding, batch_indices)                
                
                # Use Only Logit Sampled.
                one_hot = tf.stop_gradient(tf.one_hot(batch_indices,
                                                      depth = vocab_size,
                                                      dtype = tf.float32))
                logit_gen = one_hot * logits_gen
                logit_sequence.append(logit_gen)
                outputs_gen.append(output_gen)

#     return outputs_gen, logit_sequence
            

        # Discriminator Portion of GAN. 
        with tf.variable_scope('discriminator'):
            cell_dis = rnn_cell.MultiRNNCell([cell_dis] * num_layers)
            initial_state_dis = cell_dis.zero_state(batch_size, tf.float32)

# 			with tf.variable_scope('rnn'):
# 				softmax_w = tf.get_variable('softmax_w', [rnn_size, 2])
# 				softmax_b = tf.get_variable('softmax_b', [2])

# 				inputs_dis = []
# 				embedding  = tf.get_variable('embedding', [vocab_size, rnn_size])
# 				# for logit in logits_sequence:
# 				for logit in logit_sequence:
# 					inputs_dis.append(tf.matmul(logit, embedding))
					
# 				outputs_dis, last_state_dis = seq2seq.rnn_decoder(inputs_dis,
# 					initial_state_dis, cell_dis, loop_function=None)

# 			probs, logits = [], []
# 			for output_dis in outputs_dis:
# 				logit = tf.nn.xw_plus_b(output_dis, softmax_w, softmax_b)
# 				prob = tf.nn.softmax(logit)
# 				logits.append(logit)
# 				probs.append(prob)

# 			with tf.name_scope('summary'):
# 				probs      = tf.pack(probs)
# 				probs_real = tf.slice(probs, [0,0,1], [seq_length, batch_size, 1])
# 				variable_summaries(probs_real, 'probability of real')

# 			final_state_dis = last_state_dis

with tf.Session() as sess:
    outputs, logits_sequence = construct_gan()
    init_op = tf.initialize_all_variables()
    sess.run(init_op)

    for i, output in enumerate(outputs):
        print 'Iteration %d'%i
        print sess.run(output),'\n'
    
    for i, batch_logits in enumerate(logits_sequence):
        print 'Iteration %d'%i
        indices_eval = sess.run(batch_logits)
        print indices_eval


Iteration 0
[[-0.00749493  0.01697334  0.01313861  0.00229218 -0.01638157  0.00014448
  -0.00109797 -0.00095734 -0.00722192 -0.00764513]
 [-0.00749493  0.01697334  0.01313861  0.00229218 -0.01638157  0.00014448
  -0.00109797 -0.00095734 -0.00722192 -0.00764513]
 [-0.00749493  0.01697334  0.01313861  0.00229218 -0.01638157  0.00014448
  -0.00109797 -0.00095734 -0.00722192 -0.00764513]] 

Iteration 1
[[-0.02286434  0.01742898  0.03792205 -0.00479201 -0.0311817  -0.02244637
   0.00937457 -0.00589945 -0.02639428 -0.01648244]
 [-0.00574177  0.03315176  0.03856358 -0.00574849 -0.03683896  0.00802954
   0.00408992  0.01389115 -0.01845119 -0.0007752 ]
 [-0.02286434  0.01742898  0.03792205 -0.00479201 -0.0311817  -0.02244637
   0.00937457 -0.00589945 -0.02639428 -0.01648244]] 

Iteration 2
[[-0.02122686  0.02127745  0.03796593  0.00818127 -0.05512253 -0.01532887
   0.01488912  0.01152737 -0.01824609 -0.03385629]
 [-0.03725873  0.00746787  0.05718569 -0.0095898  -0.03223312 -0.01696057
   0.0227

## Random

In [13]:
indices = [['0','1','2'],['0','1','2'],['0','1','2']]

for line in indices:
    print ''.join(line)

012
012
012


In [1]:
def variable_summaries(var, name):
    '''Attach a lot of summaries to a Tensor.'''
    mean = tf.reduce_mean(var)
    tf.scalar_summary('mean/' + name, mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
    tf.scalar_summary('sttdev/' + name, stddev)
    tf.scalar_summary('max/' + name, tf.reduce_max(var))
    tf.scalar_summary('min/' + name, tf.reduce_min(var))
    tf.histogram_summary(name, var)